In [1]:
import torch

In [2]:
torch.cuda.is_available()

False

In [3]:
torch.backends.mps.is_built()

True

In [4]:
torch.backends.mps.is_available()

True

In [5]:
device = torch.device("mps")

In [6]:
import pandas as pd
import os
from torchvision.io import read_image
from torch.utils.data import Dataset

labels = pd.read_csv("labels.csv", index_col=0)

class PlayerDataset(Dataset):
    def __init__(self, labels=labels, img_dir="NBA Players"):
        self.img_labels = labels
        self.img_dir = img_dir
        self.img_files = [os.path.join(path, name) for path, subdirs, files in os.walk("NBA Players") for name in files if name.endswith("jpg")]

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_path = self.img_files[idx]
        image = read_image(img_path)
        
        player_name = img_path.split("/")[1]
        label = self.img_labels[self.img_labels["folder_name"] == player_name]["team_number"].iloc[0]

        return image, label

In [7]:
teams = labels["team"].unique()

In [8]:
len(teams)

29

In [9]:
data = PlayerDataset()

In [10]:
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_data, test_data = torch.utils.data.random_split(data, [train_size, test_size])

In [11]:
from torch.utils.data import DataLoader

BATCH_SIZE = 16

train = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

In [12]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self, classes):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.cnn = nn.Sequential(
            nn.Linear(256*256*3, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, len(classes)),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.cnn(x)
        return logits

In [13]:
model = NeuralNetwork(teams).to(device)

In [14]:
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (cnn): Sequential(
    (0): Linear(in_features=196608, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=29, bias=True)
  )
)

In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.001)

In [ ]:
size = len(train.dataset)
for batch, (images, labels) in enumerate(train):
    # Compute prediction and loss
    images = images.to(device)
    pred = model(images.float())
    labels = labels.to(device)
    loss = loss_fn(pred, labels)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
        loss, current = loss.item(), batch * len(images)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

loss: 26.033066  [    0/ 7787]


In [ ]:
logits = model(train)
pred_probab = nn.Softmax(dim=1)(logits)
pred_labels = pred_probab.argmax(1)

In [ ]:
train.dataset[2][0].shape

In [ ]:
[t[0].shape for t in list(train.dataset) if t[0].shape[1] > 256]

In [ ]:
[os.path.join(path, name) for path, subdirs, files in os.walk("NBA Players") for name in files if name.endswith("jpg")][14]